In [1]:
from pyAudioAnalysis import audioFeatureExtraction as aFE, audioBasicIO, \
audioAnalysis,audioSegmentation,audioVisualization,audioTrainTest
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import librosa
import numpy as np
import keras
from keras import models,layers
import os
from keras.utils import to_categorical
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

/Users/luigibungaro/dsr/lib/python3.6/site-packages/pydub/utils.py:165: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)
Using TensorFlow backend.


def amplitude_in_time(wave_file,threshold_seconds,sampling_rate_Hz):
    
    """this function gives you amplitude in time with a threshold settings
    with PyAudioAnalysis data loading. """
    
    Fs, X = audioBasicIO.readAudioFile(wave_file)
    sampling_indexes = []
    data_new=[]
    np.arange(0,threshold_seconds*Fs,50)
    sampling_indexes=sampling_indexes[::-1]
    if len(X)/Fs > threshold_seconds:
        X = X[0:threshold_seconds*Fs]
        data_new=[]
        for i in sampling_indexes:
            data_new.append(X[i])
        data_new = data_new[::-1]
        data.append(data_new)

paths={'peterbertley_Audio/Atraining_murmur':0,
       'peterbertley_Audio/Atraining_normal':1,
       'peterbertley_Audio/Atraining_extrahls':2}

def create_amplitude_dataset(path_dict):
    """ To create a dataset of amplitudes of the same length"""
    for k,i in enumerate(paths.keys()):
        for audio_file in os.listdir(i):
            if audio_file.endswith(".wav"):
                amplitude_in_time(i+'/'+str(audio_file),5,50)
                target.append(list(paths.values())[k]) 

data=[]
target=[]
create_amplitude_dataset(paths)

# Data --> Data --> with amplitude (vector for sample)

In [ ]:
target=[]
i=0
data=[]

for item in os.listdir('peterbertley_Audio/Atraining_murmur'):  #Atraining_normal   #Atraining_extrahls
     if item.endswith(".wav"):
            Fs, X = audioBasicIO.readAudioFile('peterbertley_Audio/Atraining_murmur/'+str(item))
            data.append(X)
            target.append(['0','murmur'])
    
for item in os.listdir('peterbertley_Audio/Atraining_normal'):    #Atraining_extrahls
     if item.endswith(".wav"):
            Fs, X = audioBasicIO.readAudioFile('peterbertley_Audio/Atraining_normal/'+str(item))
            data.append(X)
            target.append(['1','normal'])

for item in os.listdir('peterbertley_Audio/Atraining_extrahls'):   
     if item.endswith(".wav"):
            Fs, X = audioBasicIO.readAudioFile('peterbertley_Audio/Atraining_extrahls/'+str(item))
            data.append(X)
            target.append(['2','extrahls'])

## Preprocessing

In [ ]:
threshold = 5*Fs   #5 seconds threshols
sampling_indexes = np.arange(0,threshold,50)
drop=[]
for k,i in enumerate(data):
    if len(i)<threshold:
        drop.append(k)
drop = drop[::-1]  #reversing the order for the dropping
print('shape before dropping:',len(data),len(target))
for i in drop:
    data.pop(i)
    target.pop(i)
print('shape after dropping:',len(data),len(target))

# Making all the arrays of the same lengths, and sampling every 50 Hertz

for k,i in enumerate(data):
    data[k]=i[:threshold]
    data[k]=i[sampling_indexes]
data = np.array(data)

# StandardScaler
scaler = StandardScaler()
data = scaler.fit_transform(data)

print('final shape of your data is: ',data.shape)

#Setting the target with categorical transformation

target = np.array(target)
target = target[:,0]

target = to_categorical(target)
print('final categorical target shape is:',target.shape)

# Splitting data in training and test set

X_train,X_test,y_train,y_test = train_test_split(data,target,test_size=0.4)

# Data --> Feature Extraction with FeatureExtraction -  STFT (matrix for sample)

In [2]:
target=[]
i=0
data=[]

for item in os.listdir('peterbertley_Audio/Atraining_murmur'):  #Atraining_normal   #Atraining_extrahls
     if item.endswith(".wav"):
            Fs, X = audioBasicIO.readAudioFile('peterbertley_Audio/Atraining_murmur/'+str(item))
            F, f_names = aFE.stFeatureExtraction(X, Fs, 0.050*Fs, 0.025*Fs)
            data.append(F)
            target.append(['0','murmur'])
    
for item in os.listdir('peterbertley_Audio/Atraining_normal'):    #Atraining_extrahls
     if item.endswith(".wav"):
            Fs, X = audioBasicIO.readAudioFile('peterbertley_Audio/Atraining_normal/'+str(item))
            F, f_names = aFE.stFeatureExtraction(X, Fs, 0.050*Fs, 0.025*Fs)
            data.append(F)
            target.append(['1','normal'])

for item in os.listdir('peterbertley_Audio/Atraining_extrahls'):   
     if item.endswith(".wav"):
            Fs, X = audioBasicIO.readAudioFile('peterbertley_Audio/Atraining_extrahls/'+str(item))
            F, f_names = aFE.stFeatureExtraction(X, Fs, 0.050*Fs, 0.025*Fs)
            data.append(F)
            target.append(['2','extrahls'])

# Data Preprocessing

In [15]:
#Defining the threshold
print('samples are ',len(data))
list_lengths =[]
for i in data:
    list_lengths.append(i.shape[1])
threshold  =int(np.max(list_lengths)*.75)
print('The threshold is',threshold,'and represent the number of measurement for the every single feature', 
   'of the 32 features for each sample')

samples are  84
The threshold is 269 and represent the number of measurement for the every single feature of the 32 features for each sample


In [67]:
drop=[]
for k,i in enumerate(data):
    if i.shape[1] < threshold:
            drop.append(k)
drop = drop[::-1]  #reversing the order for the dropping
print('shape before dropping:',len(data),len(target))

for i in drop:
    data.pop(i)
    target.pop(i)
print('shape after dropping:',len(data),len(target))

# Making all the arrays of the same lengths, and sampling every 50 Hertz
for k,i in enumerate(data):
    data[k]=i[:,:threshold]
data = np.array(data)
print('final data shape is',data.shape)

#The features are already standardized (Below example code for checking it)
"""
pd.DataFrame(data[0].reshape(269,34),columns=f_names)
"""

#Setting the target with categorical transformation

target = np.array(target)
target = target[:,0]

target = to_categorical(target)
print('final categorical target shape is:',target.shape)

# Splitting data in training and test set

X_train,X_test,y_train,y_test = train_test_split(data,target,test_size=0.4)

shape before dropping: 70 70
shape after dropping: 70 70
final data shape is (70, 34, 269)
final categorical target shape is: (70, 3)


# ANN - Feed Forward - Model Architecture

In [ ]:
model = models.Sequential()
model.add(layers.Dense(2000, activation='relu',input_shape=(len(indexes),)))
model.add(layers.Dense(1000, activation='relu'))
model.add(layers.Dense(500, activation='relu'))
model.add(layers.Dense(200, activation='relu'))
model.add(layers.Dense(3,activation='softmax'))
model.summary()

In [ ]:
model.compile(optimizer='rmsprop',loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(data,target,test_size=0.4)

In [ ]:
history = model.fit(
    X_train,y_train,
    epochs=20,
    batch_size=256)

In [ ]:
model.evaluate(X_test,y_test)

# Stacked LSTM for sequence classification

In [ ]:
print(X_train.shape,y_train.shape)
X_train = X_train.reshape(X_train.shape[0],1,X_train.shape[1])

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Flatten, Dropout
import numpy as np

# expected input data shape: (batch_size, timesteps, data_dim)
model = Sequential()

model.add(LSTM(1000,input_shape=(1,4410,),return_sequences=True))
model.add(LSTM(500,activation='relu'))  
#model.add(Flatten())
model.add(Dense(3, activation='softmax'))


model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train,epochs=10)   #,batch_size=3

In [ ]:
print(X_test.shape)
X_test = X_test.reshape(X_test.shape[0],1,X_test.shape[1])
model.evaluate(X_test,y_test)

# Neural Network with Feature extraction

In [ ]:
import os
target=[]
i=0
data=[]


for item in os.listdir('peterbertley_Audio/Atraining_murmur'):  #Atraining_normal   #Atraining_extrahls
     if item.endswith(".wav"):
            Fs, X = audioBasicIO.readAudioFile('peterbertley_Audio/Atraining_murmur/'+str(item))
            F, f_names = aFE.stFeatureExtraction(X, Fs, 0.050*Fs, 0.025*Fs)
            data.append(F)
            target.append(['0','murmur'])
    
for item in os.listdir('peterbertley_Audio/Atraining_normal'):    #Atraining_extrahls
     if item.endswith(".wav"):
            Fs, X = audioBasicIO.readAudioFile('peterbertley_Audio/Atraining_normal/'+str(item))
            F, f_names = aFE.stFeatureExtraction(X, Fs, 0.050*Fs, 0.025*Fs)
            data.append(F)
            target.append(['1','normal'])

for item in os.listdir('peterbertley_Audio/Atraining_extrahls'):   
     if item.endswith(".wav"):
            Fs, X = audioBasicIO.readAudioFile('peterbertley_Audio/Atraining_extrahls/'+str(item))
            F, f_names = aFE.stFeatureExtraction(X, Fs, 0.050*Fs, 0.025*Fs)
            data.append(F)
            target.append(['2','extrahls'])

In [ ]:
indexes2=[]
for k,i in enumerate(data):
    if len(i[0])<240:
        indexes2.append(k)
indexes2=indexes2[::-1]
indexes2

In [ ]:
for i in indexes2:
    data.pop(i)
    target.pop(i)
print(len(data),len(target))

In [ ]:
data[0].shape

In [ ]:
for k,i in enumerate(data):
    data[k]=i[:,0:240]

In [ ]:
data = np.array(data)
data.shape

In [ ]:
target=np.array(target)
target = to_categorical(target[:,0])
X_train,X_test,y_train,y_test = train_test_split(data,target,test_size=0.4)

In [ ]:
# expected input data shape: (batch_size, timesteps, data_dim)
model = Sequential()

model.add(LSTM(500,input_shape=(34,240),return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(300,activation='relu',return_sequences=True))    
model.add(LSTM(100,activation='relu',return_sequences=True))
model.add(Flatten())
model.add(Dense(3, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train,batch_size=4,epochs=30)   #,batch_size=3

In [ ]:
model.evaluate(X_test,y_test)

In [ ]:
model.save('lstm1.h5')   #del model  (deletes the existing model)  #model = load_model('my_model.h5')

# returns a compiled model
# identical to the previous one
